In [1]:
import pandas as pd
import pandas_datareader.data as web
import re
import sqlite3
import datetime
from datetime import datetime
import numpy as np
import math
import time
import calendar
import json
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
con = sqlite3.connect('news_sentiment.db')
news_sentiment_data = pd.read_sql('select publish_date, sent_score_adj from news_sentiment', con)
con.close()

In [3]:
def MonthCharToNum(month):
    if month == 'Jan':
        return '01'
    if month == 'Feb':
        return '02'
    if month == 'Mar':
        return '03'
    if month == 'Apr':
        return '04'
    if month == 'May':
        return '05'
    if month == 'Jun':
        return '06'
    if month == 'Jul':
        return '07'
    if month == 'Aug':
        return '08'
    if month == 'Sep':
        return '09'
    if month == 'Oct':
        return '10'
    if month == 'Nov':
        return '11'
    if month == 'Dec':
        return '12'

In [4]:
## 날짜 가공
# format 1: 2021-04-12T10:11:31Z
# format 2: Fri, 9 Apr 2021 03:37:42
# format 3: Fri, 09 Apr 2021 03:37:42 

# 숫자만 나오도록 해서 형식 1 체크
ptype1 = re.compile('[0-9]+')
# 알파벳 3개 - 아무거나(쉼표인데 아무거나로 합시다) - 공백 - 숫자 1개 - 공백
ptype2 = re.compile('[a-zA-Z+][\s\S]*[\s][0-9][\s]')
# 알파벳 3개 - 아무거나(쉼표인데 아무거나로 합시다) - 공백 - 숫자 2개 - 공백
ptype3 = re.compile('[a-zA-z+][\s\S]*[\s][0-9][0-9][\s]')

for i in range(len(news_sentiment_data)):
    target = news_sentiment_data.iloc[i,0]
    
    # 맨 앞 4글자가 숫자(연도)인 format 1 (2021-04-12T10:11:31Z)
    m1 = ptype1.match(target[:4])
    # 맨 앞 7글자가 Fri, 9 (Fri 쉼표 공백 숫자 공백) 형태
    m2 = ptype2.match(target[:7])
    # 맨 앞 8글자가 Fri, 09 (Fri 쉼표 공백 숫자 숫자 공백) 형태
    m3 = ptype3.match(target[:8])
    
    if m1:
        # convert format 1 to YYYY-MM-DD string
        news_sentiment_data.iloc[i,0] = target[:10]
    
    if m2:
        # convert format 2 to YYYY-MM-DD string
        news_sentiment_data.iloc[i,0]=target[11:15]+'-'+MonthCharToNum(target[7:10])+'-'+'0'+target[5]
    
    if m3:
        # convert format 3 to YYYY-MM-DD string
        news_sentiment_data.iloc[i,0]=target[12:16]+'-'+MonthCharToNum(target[8:11])+'-'+target[5:7]

news_sentiment_data

,publish_date,sent_score_adj
0,2021-04-12,0.0661446585654584
1,2021-04-12,-0.0571803422779111
2,2021-04-12,-0.0328605479943096
3,2021-04-12,-0.0018699822946862
4,2021-04-12,0.0772847017587019
...,...,...
14444,2021-04-21,0.0297200998221272
14445,2021-04-21,0.0292613699990356
14446,2021-04-21,-0.0875424340177547
14447,2021-04-21,-0.0616340430863344


In [5]:
news_sentiment_data = news_sentiment_data.astype({'sent_score_adj' : 'float'})
periodic_score = news_sentiment_data.groupby(['publish_date'], as_index=False).mean()
periodic_score['sent_score_adj_avg5'] = periodic_score['sent_score_adj'].ewm(span=5).mean()
periodic_score['sent_score_adj_avg20'] = periodic_score['sent_score_adj'].ewm(span=20).mean()
periodic_score['sent_score_adj_avg60'] = periodic_score['sent_score_adj'].ewm(span=60).mean()
periodic_score['sent_score_adj_avg180'] = periodic_score['sent_score_adj'].ewm(span=180).mean()
periodic_score['sent_score_adj_avg360'] = periodic_score['sent_score_adj'].ewm(span=360).mean()
periodic_score

,publish_date,sent_score_adj,sent_score_adj_avg5,sent_score_adj_avg20,sent_score_adj_avg60,sent_score_adj_avg180,sent_score_adj_avg360
0,2011-06-17,-0.015912,-0.015912,-0.015912,-0.015912,-0.015912,-0.015912
1,2011-06-23,0.109856,0.059548,0.050116,0.048020,0.047321,0.047146
2,2011-07-18,0.031397,0.046214,0.043242,0.042293,0.041954,0.041867
3,2011-07-19,0.014950,0.033227,0.035075,0.035112,0.035090,0.035082
4,2011-08-25,0.190768,0.093705,0.072736,0.068352,0.066921,0.066566
...,...,...,...,...,...,...,...
291,2021-04-18,0.001208,0.001298,-0.004740,-0.013635,-0.012415,-0.009532
292,2021-04-19,-0.000747,0.000616,-0.004360,-0.013212,-0.012281,-0.009471
293,2021-04-20,0.004499,0.001910,-0.003516,-0.012632,-0.012088,-0.009375
294,2021-04-21,0.010006,0.004609,-0.002228,-0.011889,-0.011835,-0.009242


In [6]:
con = sqlite3.connect('news_periodic_score.db')
# 텍스트별로 점수 표기된 것 가져와서 평균 집계하는 거는 얼마 안 걸리니 그냥 replace 쳐도 될 것 같음
periodic_score.to_sql('news_periodic_score', con, if_exists='replace', index_label='id')
con.close()

In [44]:
periodic_score = periodic_score.set_index('publish_date')

periodic_score

,sent_score_adj,sent_score_adj_avg5,sent_score_adj_avg20,sent_score_adj_avg60,sent_score_adj_avg180,sent_score_adj_avg360
publish_date,,,,,,
2011-06-17,-0.015912,-0.015912,-0.015912,-0.015912,-0.015912,-0.015912
2011-06-23,0.109856,0.059548,0.050116,0.048020,0.047321,0.047146
2011-07-18,0.031397,0.046214,0.043242,0.042293,0.041954,0.041867
2011-07-19,0.014950,0.033227,0.035075,0.035112,0.035090,0.035082
2011-08-25,0.190768,0.093705,0.072736,0.068352,0.066921,0.066566
...,...,...,...,...,...,...
2021-04-18,0.001208,0.001298,-0.004740,-0.013635,-0.012415,-0.009532
2021-04-19,-0.000747,0.000616,-0.004360,-0.013212,-0.012281,-0.009471
2021-04-20,0.004499,0.001910,-0.003516,-0.012632,-0.012088,-0.009375


In [57]:
# start보다 1거래일 전까지 가져옴
start = '2021-04-09'
#end = datetime.today().strftime('%Y-%m-%d')
end = '2021-04-21'

sp500 = web.DataReader('^GSPC', 'yahoo', start, end)['Adj Close'].to_frame("S&P 500 Close")
sp500 = sp500.loc[start:]

# 빈 날짜를 직전 거래일 값으로 채워야 됨
sp500_adj = sp500.resample('D').first()
sp500_adj = sp500_adj.fillna(method='pad')
sp500_adj

,S&P 500 Close
Date,
2021-04-09,4128.799805
2021-04-10,4128.799805
2021-04-11,4128.799805
2021-04-12,4127.990234
2021-04-13,4141.589844
2021-04-14,4124.660156
2021-04-15,4170.419922
2021-04-16,4185.470215
2021-04-17,4185.470215


In [58]:
periodic_score.loc[start:end]

,sent_score_adj,sent_score_adj_avg5,sent_score_adj_avg20,sent_score_adj_avg60,sent_score_adj_avg180,sent_score_adj_avg360
publish_date,,,,,,
2021-04-09,0.001278,-0.004599,-0.012793,-0.018594,-0.013830,-0.010176
2021-04-10,0.003989,-0.001736,-0.011195,-0.017853,-0.013624,-0.010078
2021-04-11,-0.006896,-0.003456,-0.010786,-0.017494,-0.013547,-0.010055
2021-04-12,-0.007210,-0.004707,-0.010445,-0.017157,-0.013473,-0.010036
2021-04-13,0.001753,-0.002554,-0.009283,-0.016537,-0.013298,-0.009954
2021-04-14,0.005291,0.000061,-0.007895,-0.015821,-0.013084,-0.009848
2021-04-15,0.003208,0.001110,-0.006838,-0.015197,-0.012896,-0.009757
2021-04-16,-0.000499,0.000574,-0.006234,-0.014715,-0.012754,-0.009693
2021-04-17,0.002882,0.001343,-0.005366,-0.014138,-0.012574,-0.009606


In [59]:
# 그림 그려야 됨
sp500_adj['Sentiment_1D'] = list(periodic_score['sent_score_adj'].loc[start:end])
sp500_adj['Sentiment_5D'] = list(periodic_score['sent_score_adj_avg5'].loc[start:end])
sp500_adj['Sentiment_20D'] = list(periodic_score['sent_score_adj_avg20'].loc[start:end])
sp500_adj['Sentiment_60D'] = list(periodic_score['sent_score_adj_avg60'].loc[start:end])

sp500_adj

,S&P 500 Close,Sentiment_1D,Sentiment_5D,Sentiment_20D,Sentiment_60D
Date,,,,,
2021-04-09,4128.799805,0.001278,-0.004599,-0.012793,-0.018594
2021-04-10,4128.799805,0.003989,-0.001736,-0.011195,-0.017853
2021-04-11,4128.799805,-0.006896,-0.003456,-0.010786,-0.017494
2021-04-12,4127.990234,-0.007210,-0.004707,-0.010445,-0.017157
2021-04-13,4141.589844,0.001753,-0.002554,-0.009283,-0.016537
2021-04-14,4124.660156,0.005291,0.000061,-0.007895,-0.015821
2021-04-15,4170.419922,0.003208,0.001110,-0.006838,-0.015197
2021-04-16,4185.470215,-0.000499,0.000574,-0.006234,-0.014715
2021-04-17,4185.470215,0.002882,0.001343,-0.005366,-0.014138


In [60]:
# secondary y-axis
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_1D'], name='News Sentiment 1D'), secondary_y=False,)
fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['S&P 500 Close'], name='S&P 500 Close'), secondary_y=True,)
fig.update_layout(title_text='News Sentiment and S&P 500 Index Close')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text="News Sentiment", secondary_y=False)
fig.update_yaxes(title_text="S&P 500 Close", secondary_y=True)

fig.show()

In [61]:
# start보다 1거래일 전까지 가져옴
# 5일 이평 데이터가 4월 13일부터 정상임
start = '2021-04-13'
#end = datetime.today().strftime('%Y-%m-%d')
end = '2021-04-21'

sp500 = web.DataReader('^GSPC', 'yahoo', start, end)['Adj Close'].to_frame("S&P 500 Close")
sp500 = sp500.loc[start:]

# 빈 날짜를 직전 거래일 값으로 채워야 됨
sp500_adj = sp500.resample('D').first()
sp500_adj = sp500_adj.fillna(method='pad')
sp500_adj

,S&P 500 Close
Date,
2021-04-13,4141.589844
2021-04-14,4124.660156
2021-04-15,4170.419922
2021-04-16,4185.470215
2021-04-17,4185.470215
2021-04-18,4185.470215
2021-04-19,4163.259766
2021-04-20,4134.939941
2021-04-21,4173.419922


In [62]:
# 그림 그려야 됨
sp500_adj['Sentiment_1D'] = list(periodic_score['sent_score_adj'].loc[start:end])
sp500_adj['Sentiment_5D'] = list(periodic_score['sent_score_adj_avg5'].loc[start:end])
sp500_adj['Sentiment_20D'] = list(periodic_score['sent_score_adj_avg20'].loc[start:end])
sp500_adj['Sentiment_60D'] = list(periodic_score['sent_score_adj_avg60'].loc[start:end])

sp500_adj

,S&P 500 Close,Sentiment_1D,Sentiment_5D,Sentiment_20D,Sentiment_60D
Date,,,,,
2021-04-13,4141.589844,0.001753,-0.002554,-0.009283,-0.016537
2021-04-14,4124.660156,0.005291,0.000061,-0.007895,-0.015821
2021-04-15,4170.419922,0.003208,0.001110,-0.006838,-0.015197
2021-04-16,4185.470215,-0.000499,0.000574,-0.006234,-0.014715
2021-04-17,4185.470215,0.002882,0.001343,-0.005366,-0.014138
2021-04-18,4185.470215,0.001208,0.001298,-0.004740,-0.013635
2021-04-19,4163.259766,-0.000747,0.000616,-0.004360,-0.013212
2021-04-20,4134.939941,0.004499,0.001910,-0.003516,-0.012632
2021-04-21,4173.419922,0.010006,0.004609,-0.002228,-0.011889


In [63]:
# secondary y-axis
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_5D'], name='News Sentiment 5D'), secondary_y=False,)
#fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_20D'], name='News Sentiment 20D'), secondary_y=False,)
#fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_60D'], name='News Sentiment 60D'), secondary_y=False,)
fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['S&P 500 Close'], name='S&P 500 Close'), secondary_y=True,)
fig.update_layout(title_text='News Sentiment Moving Average and S&P 500 Index Close')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text="News Sentiment", secondary_y=False)
fig.update_yaxes(title_text="S&P 500 Close", secondary_y=True)

fig.show()

In [64]:
# secondary y-axis
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_1D'], name='News Sentiment 1D'), secondary_y=False,)
fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_5D'], name='News Sentiment 5D'), secondary_y=False,)
#fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_20D'], name='News Sentiment 20D'), secondary_y=False,)
#fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['Sentiment_60D'], name='News Sentiment 60D'), secondary_y=False,)
fig.add_trace(go.Scatter(x=list(sp500_adj.index), y=sp500_adj['S&P 500 Close'], name='S&P 500 Close'), secondary_y=True,)
fig.update_layout(title_text='News Sentiment Moving Average and S&P 500 Index Close')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text="News Sentiment", secondary_y=False)
fig.update_yaxes(title_text="S&P 500 Close", secondary_y=True)

fig.show()